## Imports

In [144]:
import pandas as pd
import numpy as np
import json
import os
from tqdm import tqdm
tqdm.pandas()


import matplotlib.pyplot as plt
from datetime import datetime



In [145]:
with open('../config.json', 'r') as file:
    config = json.load(file)

In [154]:
supervised_df = pd.DataFrame([])

In [155]:
for filename in os.listdir(config['extracted_file_path']):
    if 'gz'not in filename:
        continue
    file_path = os.path.join(config['extracted_file_path'], filename)
    
    # Check if it is a file (and not a directory)
    if os.path.isfile(file_path):
        print(file_path)
        dirty_df = pd.read_csv(file_path)
# dirty_df = pd.read_csv(f"{config['extracted_file_path']}/season-df-4-40001-43000.csv.gz")
        df = cleanup(dirty_df)
        temp_df = df[['Success','TankClass','HealerClass','DpsClass','NumberOfDeaths','TotalTimeTaken','dungeon.name']].copy()
        supervised_df = pd.concat([supervised_df, temp_df], ignore_index=True)


/Users/sarahcarr/Projects/python/mythic_ETL/extract/extracted/season-df-4-36001-40000.csv.gz
/Users/sarahcarr/Projects/python/mythic_ETL/extract/extracted/season-df-4-20001-23000.csv.gz
/Users/sarahcarr/Projects/python/mythic_ETL/extract/extracted/season-df-4-79001-80001.csv.gz
/Users/sarahcarr/Projects/python/mythic_ETL/extract/extracted/season-df-4-68002-69002.csv.gz
/Users/sarahcarr/Projects/python/mythic_ETL/extract/extracted/season-df-4-103001-106000.csv.gz
/Users/sarahcarr/Projects/python/mythic_ETL/extract/extracted/season-df-4-84001-85001.csv.gz
/Users/sarahcarr/Projects/python/mythic_ETL/extract/extracted/season-df-4-72001-73001.csv.gz
/Users/sarahcarr/Projects/python/mythic_ETL/extract/extracted/season-df-4-73001-74001.csv.gz
/Users/sarahcarr/Projects/python/mythic_ETL/extract/extracted/season-df-4-98001-99001.csv.gz
/Users/sarahcarr/Projects/python/mythic_ETL/extract/extracted/season-df-4-100001-103000.csv.gz
/Users/sarahcarr/Projects/python/mythic_ETL/extract/extracted/seas

SyntaxError: invalid syntax. Perhaps you forgot a comma? (<string>, line 1)

# Data Cleanup

In [77]:
def extract_tank_class(x):
    for r in eval(x):
        if r['character']['spec']['role'] == 'tank':
            return r['character']['class']['slug']


In [78]:
def extract_healer_class(x):
    for r in eval(x):
        if r['character']['spec']['role'] == 'healer':
            return r['character']['class']['slug']

In [79]:
def extract_dps_class(x):
    class_list = []
    for r in eval(x):
        if r['character']['spec']['role'] == 'dps':
            class_list.append(r['character']['class']['slug'])

    class_list.sort()
    return class_list

In [106]:
def extract_success(x):
    if x['time_remaining_ms'] > 0:
        encounters = eval(x['logged_details.encounters'])
        if len(encounters) >= x['dungeon.num_bosses']:
            counter = 0
            for enc in encounters:
                if enc['is_success']:
                    counter+=1
            if counter >= x['dungeon.num_bosses']:
                return True
    return False

In [81]:
def calculate_num_deaths(x):
    if x == []:
        return 0
    elif type(x) == str:
        return len(eval(x))
    else:
        return 0

In [149]:
def cleanup(dataframe):
    df = dataframe.dropna(subset=['logged_details.enemies', 'logged_details.encounters', 'logged_details.deaths']).copy()
    df['TankClass'] = df['roster'].apply(extract_tank_class)
    df['HealerClass'] = df['roster'].apply(extract_healer_class)
    df['DpsClass'] = df['roster'].apply(extract_dps_class)
    df['Success'] = df.apply(extract_success, axis=1)
    df['NumberOfDeaths'] = df['logged_details.deaths'].apply(calculate_num_deaths)
    df['TotalTimeTaken'] = df['clear_time_ms']/60000
    return df


## Supervised Learning
Success, TankClass, HealerClass, DpsClass, NumberOfDeaths, TotalTimeTaken, dungeon.name

In [156]:
supervised_df.count()

Success           24739
TankClass         24739
HealerClass       24723
DpsClass          24739
NumberOfDeaths    24739
TotalTimeTaken    24739
dungeon.name      24739
dtype: int64

In [157]:
supervised_df.describe()

,NumberOfDeaths,TotalTimeTaken
count,24739.000000,24739.000000
mean,7.881644,26.095941
std,7.911011,7.277763
min,0.000000,9.898483
25%,2.000000,21.024067
50%,6.000000,24.968750
75%,11.000000,29.798800
max,108.000000,108.676750


In [158]:
supervised_df.head()

,Success,TankClass,HealerClass,DpsClass,NumberOfDeaths,TotalTimeTaken,dungeon.name
0,True,paladin,priest,"[demon-hunter, hunter, warlock]",2,21.138500,Halls of Infusion
1,True,demon-hunter,druid,"[monk, paladin, paladin]",4,19.361667,Halls of Infusion
2,True,demon-hunter,druid,"[hunter, monk, paladin]",3,20.258850,Algeth'ar Academy
3,True,paladin,druid,"[mage, shaman, warlock]",1,24.897383,Uldaman: Legacy of Tyr
4,True,demon-hunter,druid,"[demon-hunter, evoker, mage]",4,15.684633,Ruby Life Pools


In [159]:
config['cleaned_supervised']

'/Users/sarahcarr/Projects/python/mythic_ETL/transform/cleaned_supervised'

In [161]:
supervised_df.to_csv(config['cleaned_supervised'] + '/supervised_data.csv.gz', index=False, compression='gzip')